# 0. Imports and Setting up Anthropic API Client

In [6]:
from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install python-dotenv

import os
import dotenv

dotenv.load_dotenv('/content/drive/MyDrive/.env')

True

In [3]:
# Load Prompts and Problem Description
prompt1_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt1_MathematicalModel.txt'
prompt2_path = '/content/drive/MyDrive/Thesis/Prompts/Prompt2_PyomoCode.txt'
problem_desc_path = '/content/drive/MyDrive/Thesis/ProblemDescriptions/IP/IP2.txt'

prompt1_file = open(prompt1_path, "r")
prompt2_file = open(prompt2_path, "r")
problem_desc_file = open(problem_desc_path, "r")

prompt1 = prompt1_file.read()
print("Prompt 1:\n", prompt1)

prompt2 = prompt2_file.read()
print("Prompt 2:\n", prompt2)

problem_desc = problem_desc_file.read()
print("Problem Description:\n", problem_desc)

Prompt 1:
 Please write a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer.
Prompt 2:
 Please write a python pyomo code for this optimization problem.
Use sample data where needed.
Indicate where you use sample data.
Problem Description:
 Your goal is to invest in several of 10 possible investment strategies in the most optimal way. The historic returns of those strategies are stored in the file "investments_data.csv". Each column represents one strategy and the rows are the past investment outcomes. There is no index and the values are separated by a ;.

The costs for investing in a given investment is stored in a vector A, which has one value for each strategy in order.  
The values are: [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]

You can only invest once into an investment. 

Unfortunately due to other costs and inflation, your available budget at this time is uncertain. There are

In [4]:
!pip install anthropic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.7/877.7 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.7 MB/s eta 0:00:00


In [5]:
# Importing Anthropic & Setting Headers
import anthropic

client = anthropic.Anthropic(
    # defaults to os.environ.get("ANTHROPIC_API_KEY")
    api_key=os.environ.get("ANTHROPIC_API_KEY"),
)

# 1. Prompt 1: Create Mathematical Model

In [6]:
message = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt1,
    max_tokens=4096,
    messages=[
        {"role": "user", "content": problem_desc}
    ]
)

response_p1 = message.content[0].text

In [7]:
# Print response
print(response_p1)

To formulate the mean-variance optimization model for this problem, we need to define the parameters, decision variables, objective function, and constraints. Let's first define the parameters and decision variables.

Parameters:
- n: number of investment strategies (n = 10)
- m: number of past investment outcomes for each strategy
- R_ij: past investment outcome for strategy i in period j (i = 1, ..., n; j = 1, ..., m)
- A_i: cost for investing in strategy i (i = 1, ..., n)
- r: risk parameter (r = 2)
- B_k: available budget in scenario k (k = 1, ..., 4)
- p_k: probability of scenario k (k = 1, ..., 4)
- α: tolerable probability of exceeding the budget (α = 0.4)

Decision Variables:
- x_i: binary variable indicating whether to invest in strategy i (x_i = 1) or not (x_i = 0) (i = 1, ..., n)

Objective Function:
The objective is to maximize the expected return while minimizing the risk, which can be formulated as:
max ∑(i=1 to n) (1/m × ∑(j=1 to m) R_ij) × x_i - r × sqrt(∑(i=1 to n) (1/

# 2. Prompt 2: Write Code based on Mathematical Model

In [8]:
message2 = client.messages.create(
    model="claude-3-opus-20240229",
    system=prompt2,
    max_tokens=4069,
    messages=[
        {"role": "user", "content": response_p1}
    ]
)

response_p2 = message2.content[0].text

In [9]:
# Print response
print(response_p2)

Here's a Python code using Pyomo to solve the mean-variance optimization problem:

```python
import pyomo.environ as pyo
import numpy as np

# Parameters
n = 10  # number of investment strategies
m = 5   # number of past investment outcomes for each strategy
r = 2   # risk parameter
alpha = 0.4  # tolerable probability of exceeding the budget

# Sample data (replace with actual data)
R = np.random.rand(n, m)  # past investment outcomes
A = np.random.rand(n)     # cost for investing in each strategy
B = np.array([10, 12, 15, 18])  # available budget in each scenario
p = np.array([0.2, 0.3, 0.3, 0.2])  # probability of each scenario

# Create the Pyomo model
model = pyo.ConcreteModel()

# Decision variables
model.x = pyo.Var(range(n), within=pyo.Binary)

# Objective function
model.obj = pyo.Objective(expr=sum((1/m * sum(R[i,j] for j in range(m))) * model.x[i] for i in range(n))
                          - r * pyo.sqrt(sum((1/m * sum((R[i,j] - (1/m * sum(R[i,j] for j in range(m))))**2 for

# 4. Input Problem Data and Test Model Code

In [4]:
# Installing pyomo nd solver
!pip install -q pyomo
!pip install pandas
!apt-get install -y -qq glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 74.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 6.0 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 121918 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected pac

In [20]:
import pyomo.environ as pyo
import pandas as pd

# Parameters
n = 10  # number of investment strategies
m = 200   # number of past investment outcomes for each strategy
r = 2   # risk parameter
alpha = 0.4  # tolerable probability of exceeding the budget

# Sample data (replace with actual data)
past_investments = pd.read_csv("/content/drive/MyDrive/Thesis/ProblemData/IP/IP2/investments_data.csv", header=None, sep=';')
R = pd.DataFrame(past_investments).to_numpy()
R = R.T
A = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]  # cost for investing in each strategy
B = [1000, 1100, 900, 1200]  # available budget in each scenario
p = [0.55, 0.4, 0.04, 0.01]  # probability of each scenario

# Create the Pyomo model
model = pyo.ConcreteModel()

# Decision variables
model.x = pyo.Var(range(n), within=pyo.Binary)

# Objective function
model.obj = pyo.Objective(expr=sum((1/m * sum(R[i,j] for j in range(m))) * model.x[i] for i in range(n))
                          - r * pyo.sqrt(sum((1/m * sum((R[i,j] - (1/m * sum(R[i,j] for j in range(m))))**2 for j in range(m))) * model.x[i] for i in range(n))),
                          sense=pyo.maximize)

# Constraints
def budget_constraint(model, k):
    return pyo.summation(A, model.x) <= B[k]

model.budget = pyo.Constraint(range(len(B)), rule=budget_constraint)

def probability_constraint(model):
    return sum(p[k] * model.budget[k].body for k in range(len(B))) >= 1 - alpha

model.probability = pyo.Constraint(rule=probability_constraint)

# Solve the model
solver = pyo.SolverFactory('glpk')
results = solver.solve(model)

# Print the results
print("Objective value:", pyo.value(model.obj))
print("Selected strategies:")
for i in range(n):
    if pyo.value(model.x[i]) == 1:
        print("Strategy", i+1)

ValueError: Model objective (obj) contains nonlinear terms that cannot be written to LP format

# 5. Correct The Model Code to Test Mathematical Model (if applicable)

In [13]:
# Installing pyomo nd solver
import sys
import os

if 'google.colab' in sys.modules:
    !pip install idaes-pse --pre
    !idaes get-extensions --to ./bin
    os.environ['PATH'] += ':bin'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.9/17.9 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.0/305.0 kB 26.0 MB/s eta 0:00:00
future: the set contained only NumericConstant and _PythonCallbackFunctionID,
and provided no meaningful value to clients or walkers.  Users should likely
handle these types in the same manner as immutable Params.  (deprecated in
6.7.2) (called from <frozen importlib._bootstrap>:241)
'_BlockData'.  The class '_BlockData' has been renamed to 'BlockData'.
(deprecated in 6.7.2) (called from /usr/local/lib/python3.10/dist-
packages/idaes/core/base/process_base.py:78)
from '_GeneralExpressionData'.  The class '_GeneralExpressionData' has been
renamed to 'ExpressionData'.  (deprecated in 6.7.2) (called from
/usr/local/lib/python3.10/dist-
packages/idaes/core/base/var_like_expression.py:31)
Getting files...
Done
---------------------------------------------------
IDAES Extensions Build Versions
Solvers:  vno version file found
Libr

In [21]:
import pyomo.environ as pyo
import pandas as pd

# Parameters
n = 10  # number of investment strategies
m = 200   # number of past investment outcomes for each strategy
r = 2   # risk parameter
alpha = 0.4  # tolerable probability of exceeding the budget

# Sample data (replace with actual data)
past_investments = pd.read_csv("/content/drive/MyDrive/Thesis/ProblemData/IP/IP2/investments_data.csv", header=None, sep=';')
R = pd.DataFrame(past_investments).to_numpy()
R = R.T
A = [80, 340, 410, 50, 180, 221, 15, 348, 191, 225]  # cost for investing in each strategy
B = [1000, 1100, 900, 1200]  # available budget in each scenario
p = [0.55, 0.4, 0.04, 0.01]  # probability of each scenario

# Small value to prevent sqrt from evaluating zero
epsilon = 1e-6 # MODIFIED TO INCLUDE EPSILON

# Create the Pyomo model
model = pyo.ConcreteModel()

# Decision variables
model.x = pyo.Var(range(n), within=pyo.Binary)

# Objective function
model.obj = pyo.Objective(expr=sum((1/m * sum(R[i,j] for j in range(m))) * model.x[i] for i in range(n))
                          - r * pyo.sqrt(sum((1/m * sum((R[i,j] - (1/m * sum(R[i,j] for j in range(m))))**2 for j in range(m))) * model.x[i] for i in range(n)) + epsilon), # MODIFIED TO INCLUDE EPSILON
                          sense=pyo.maximize)

# Constraints
def budget_constraint(model, k):
    return pyo.summation(A, model.x) <= B[k]

model.budget = pyo.Constraint(range(len(B)), rule=budget_constraint)

def probability_constraint(model):
    return sum(p[k] * model.budget[k].body for k in range(len(B))) >= 1 - alpha

model.probability = pyo.Constraint(rule=probability_constraint)

# Solve the model
solver = pyo.SolverFactory('couenne')
results = solver.solve(model)

# Print the results
print("Objective value:", pyo.value(model.obj))
print("Selected strategies:")
for i in range(n):
    if pyo.value(model.x[i]) == 1:
        print("Strategy", i+1)

Objective value: 217.08352160678973
Selected strategies:
Strategy 1
Strategy 6
Strategy 8
Strategy 10
